In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as la
import scipy.sparse as sparse

In [3]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt   

In [4]:
#Create grid

Nx = 6                              #dimensions
Ny = 8

x_I, x_F = 0, np.pi                  #boundary points
y_I, y_F = 0, np.pi

dx = (x_F - x_I)/Nx                  #step sizes
dy = (y_F - y_I)/Ny
step_rat = dx**2/dy**2

X = np.linspace(x_I, x_F, Nx+1)      #grid
Y = np.linspace(y_I, y_F, Ny+1)

x, y = np.meshgrid(X,Y)              #2D grid

In [5]:
#creat main diagonal block
b1 = np.ones(Nx)
b1[Nx-1] = 0

b2 = -2*(1+step_rat)*np.ones(Nx+1)
b2[0] = 1
b2[Nx] = 1

b3 = np.ones(Nx)
b3[0] = 0


B = sparse.diags([b1, b2, b3], [-1, 0, 1]).toarray()             #main block matrix

print(B)

M1 = sparse.kron(sparse.eye(Ny+1), B).toarray()                  #central component of the tridiagonal block matrix

[[ 1.    0.    0.    0.    0.    0.    0.  ]
 [ 1.   -5.56  1.    0.    0.    0.    0.  ]
 [ 0.    1.   -5.56  1.    0.    0.    0.  ]
 [ 0.    0.    1.   -5.56  1.    0.    0.  ]
 [ 0.    0.    0.    1.   -5.56  1.    0.  ]
 [ 0.    0.    0.    0.    1.   -5.56  1.  ]
 [ 0.    0.    0.    0.    0.    0.    1.  ]]


In [6]:
#creat upper and lower diagonal blocks
a = sparse.diags([np.ones(Ny), np.ones(Ny)], [-1, 1]).toarray()
a[0,1] = 2            #in case of neumann boundary conditions
a[-1,-2] = 2

A = step_rat*sparse.eye(Nx+1).toarray()                       #upper/lower block matrix
A[0, 0] = 0
A[Nx, Nx] = 0

M2 = sparse.kron(a, A).toarray()                              #upper/lower component of the tridiagonal block matrix
# print(A)

In [7]:
#creat the tridiagonal block matrix of the 2d poisson equation.
M = M1 + M2       
# print(M)

In [8]:
def u_analytical(x, y):

    uan = x*(np.pi-x)*np.cos(y)+y

    return uan

def source(x, y): 

    f = -np.cos(y)*(x*(np.pi-x)+2)

    return f

In [9]:
rhs = source(x, y)*dx**2
print(rhs, '\n')
rhs = rhs.reshape((Nx+1)*(Ny+1))
print(rhs, '\n')
rhs[0::Nx+1] = Y
rhs[Nx::Nx+1] = Y
print(rhs, '\n')
rhs[1:Nx] += 2*step_rat*dy
rhs[-Nx:-1] -= 2*step_rat*dy
print(rhs)


[[-5.48e-01 -9.24e-01 -1.15e+00 -1.22e+00 -1.15e+00 -9.24e-01 -5.48e-01]
 [-5.07e-01 -8.54e-01 -1.06e+00 -1.13e+00 -1.06e+00 -8.54e-01 -5.07e-01]
 [-3.88e-01 -6.53e-01 -8.13e-01 -8.66e-01 -8.13e-01 -6.53e-01 -3.88e-01]
 [-2.10e-01 -3.54e-01 -4.40e-01 -4.69e-01 -4.40e-01 -3.54e-01 -2.10e-01]
 [-3.36e-17 -5.66e-17 -7.04e-17 -7.50e-17 -7.04e-17 -5.66e-17 -3.36e-17]
 [ 2.10e-01  3.54e-01  4.40e-01  4.69e-01  4.40e-01  3.54e-01  2.10e-01]
 [ 3.88e-01  6.53e-01  8.13e-01  8.66e-01  8.13e-01  6.53e-01  3.88e-01]
 [ 5.07e-01  8.54e-01  1.06e+00  1.13e+00  1.06e+00  8.54e-01  5.07e-01]
 [ 5.48e-01  9.24e-01  1.15e+00  1.22e+00  1.15e+00  9.24e-01  5.48e-01]] 

[-5.48e-01 -9.24e-01 -1.15e+00 -1.22e+00 -1.15e+00 -9.24e-01 -5.48e-01
 -5.07e-01 -8.54e-01 -1.06e+00 -1.13e+00 -1.06e+00 -8.54e-01 -5.07e-01
 -3.88e-01 -6.53e-01 -8.13e-01 -8.66e-01 -8.13e-01 -6.53e-01 -3.88e-01
 -2.10e-01 -3.54e-01 -4.40e-01 -4.69e-01 -4.40e-01 -3.54e-01 -2.10e-01
 -3.36e-17 -5.66e-17 -7.04e-17 -7.50e-17 -7.04e-17 -5.66

In [10]:
u = la.solve(M, rhs)
u = u.reshape(Ny+1, Nx+1)

u_an = u_analytical(x, y)

print(u, '\n', u_an)

[[0.   1.38 2.21 2.48 2.21 1.38 0.  ]
 [0.39 1.67 2.43 2.69 2.43 1.67 0.39]
 [0.79 1.76 2.35 2.54 2.35 1.76 0.79]
 [1.18 1.71 2.02 2.13 2.02 1.71 1.18]
 [1.57 1.57 1.57 1.57 1.57 1.57 1.57]
 [1.96 1.44 1.12 1.01 1.12 1.44 1.96]
 [2.36 1.38 0.8  0.6  0.8  1.38 2.36]
 [2.75 1.47 0.71 0.45 0.71 1.47 2.75]
 [3.14 1.76 0.93 0.66 0.93 1.76 3.14]] 
 [[0.   1.37 2.19 2.47 2.19 1.37 0.  ]
 [0.39 1.66 2.42 2.67 2.42 1.66 0.39]
 [0.79 1.75 2.34 2.53 2.34 1.75 0.79]
 [1.18 1.7  2.02 2.12 2.02 1.7  1.18]
 [1.57 1.57 1.57 1.57 1.57 1.57 1.57]
 [1.96 1.44 1.12 1.02 1.12 1.44 1.96]
 [2.36 1.39 0.81 0.61 0.81 1.39 2.36]
 [2.75 1.48 0.72 0.47 0.72 1.48 2.75]
 [3.14 1.77 0.95 0.67 0.95 1.77 3.14]]


In [11]:
fig = plt.figure()                                #create 3D figure and axes
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, u, label = 'numerical', c = 'r')
ax.plot_surface(x, y, u_an, label = 'analytical')

ax.set_xlabel('x')
ax.set_ylabel('y')

# ax.legend()


Text(0.5, 0, 'y')